# Teamfights

In [1]:
import requests
import pandas as pd

from opendota import(
    query_opendota,
)

Выгружаем табличку с информацией по дракам

In [61]:
ODOTA_QUERY = ''' 
SELECT
    matches.match_id,
    matches.teamfights,
    player_matches.purchase_log,
    player_matches.player_slot
FROM 
    matches
        JOIN 
            player_matches using(match_id)
ORDER BY matches.match_id DESC NULLS LAST
LIMIT 200
'''

In [67]:
def norm_slot(player_slot):
    slot = player_slot if player_slot < 5 else player_slot - 123
    return slot

In [68]:
df_teamfights = query_opendota(ODOTA_QUERY)
df_teamfights['slot'] = df_teamfights.apply((lambda row: norm_slot(row['player_slot'])), axis=1)
df_teamfights.head()

,match_id,player_slot,purchase_log,teamfights,slot
0,3575051322,3,"[{'time': -89, 'key': 'boots'}, {'time': -89, ...","[{'start': 586, 'end': 631, 'last_death': 616,...",3
1,3575051322,4,"[{'time': -89, 'key': 'ward_observer'}, {'time...","[{'start': 586, 'end': 631, 'last_death': 616,...",4
2,3575051322,129,"[{'time': -89, 'key': 'stout_shield'}, {'time'...","[{'start': 586, 'end': 631, 'last_death': 616,...",6
3,3575051322,131,"[{'time': -89, 'key': 'tpscroll'}, {'time': -8...","[{'start': 586, 'end': 631, 'last_death': 616,...",8
4,3575051322,132,"[{'time': -89, 'key': 'tpscroll'}, {'time': -7...","[{'start': 586, 'end': 631, 'last_death': 616,...",9


В teamfights содержится информация по всем дракам в игре

Число тимфайтов в игре по мнению доты

In [53]:
len(df_teamfights.teamfights[0])

4

Структура записи по тимфайту (рассматриваем самую первую драку в игре)

In [54]:
tf = df_teamfights.teamfights[0][0]
tf.keys()

dict_keys(['start', 'end', 'last_death', 'deaths', 'players'])

Посмотреть временные границы драки

In [55]:
print(tf['start'], tf['end'])

586 631


В players содержится информация по всем 10 игрокам в матче

In [56]:
len(tf['players'])

10

In [57]:
player_info = tf['players'][1]
player_info.keys()

dict_keys(['deaths_pos', 'ability_uses', 'item_uses', 'killed', 'deaths', 'buybacks', 'damage', 'healing', 'gold_delta', 'xp_delta', 'xp_start', 'xp_end'])

Информация по игроку за период времени ```tf['start']``` по ```tf['end']```

In [58]:
player_info

{'ability_uses': {'puck_dream_coil': 1, 'puck_illusory_orb': 1},
 'buybacks': 0,
 'damage': 434,
 'deaths': 1,
 'deaths_pos': {'168': {'92': 1}},
 'gold_delta': -16,
 'healing': 0,
 'item_uses': {'tango': 1},
 'killed': {},
 'xp_delta': 95,
 'xp_end': 4671,
 'xp_start': 4576}

Я знаю границы тимфайта, как теперь понять, что купили перед дракой?

In [71]:
purchase_log = df_teamfights.purchase_log[0]

In [60]:
purchase_log

[{'key': 'boots', 'time': -89},
 {'key': 'tango', 'time': -89},
 {'key': 'clarity', 'time': -89},
 {'key': 'tpscroll', 'time': -89},
 {'key': 'flask', 'time': 179},
 {'key': 'orb_of_venom', 'time': 205},
 {'key': 'wind_lace', 'time': 277},
 {'key': 'clarity', 'time': 309},
 {'key': 'branches', 'time': 310},
 {'key': 'magic_stick', 'time': 419},
 {'key': 'branches', 'time': 420},
 {'key': 'enchanted_mango', 'time': 422},
 {'key': 'magic_wand', 'time': 441},
 {'key': 'tpscroll', 'time': 580},
 {'key': 'tpscroll', 'time': 670},
 {'key': 'ring_of_regen', 'time': 784},
 {'key': 'tranquil_boots', 'time': 785},
 {'key': 'infused_raindrop', 'time': 786},
 {'key': 'tpscroll', 'time': 787},
 {'key': 'tpscroll', 'time': 995},
 {'key': 'tpscroll', 'time': 1073},
 {'key': 'blink', 'time': 1141},
 {'key': 'tpscroll', 'time': 1159},
 {'key': 'smoke_of_deceit', 'time': 1171},
 {'key': 'tpscroll', 'time': 1293}]

In [74]:
tf = df_teamfights.teamfights[0][0]
print(tf['start'], tf['end'])

586 631


In [76]:
bought_before_fight = []
for purchase in purchase_log:
    if purchase['time'] < tf['start']:
        bought_before_fight.append(purchase)
bought_before_fight

[{'key': 'boots', 'time': -89},
 {'key': 'tango', 'time': -89},
 {'key': 'clarity', 'time': -89},
 {'key': 'tpscroll', 'time': -89},
 {'key': 'flask', 'time': 179},
 {'key': 'orb_of_venom', 'time': 205},
 {'key': 'wind_lace', 'time': 277},
 {'key': 'clarity', 'time': 309},
 {'key': 'branches', 'time': 310},
 {'key': 'magic_stick', 'time': 419},
 {'key': 'branches', 'time': 420},
 {'key': 'enchanted_mango', 'time': 422},
 {'key': 'magic_wand', 'time': 441},
 {'key': 'tpscroll', 'time': 580}]